In [1]:
import pandas as pd
import os
from typing import Tuple
from vg import create_vg_fund_info
from blk import create_blk_fund_info
from helpers import get_yahoo_finance_data, get_tipranks_ratings
from datetime import datetime

In [3]:
def fetch_most_recent_info() -> Tuple[pd.DataFrame, pd.DataFrame]:
  return create_blk_fund_info(), create_vg_fund_info()


def handle_new_day() -> Tuple[pd.DataFrame, pd.DataFrame]:
  today_date = datetime.today().strftime('%Y-%m-%d')
  out_workbooks = os.listdir("./out")
  curr_date = out_workbooks[0].split("_")[0]  
  if (datetime.strptime(today_date, "%Y-%m-%d").date() > datetime.strptime(curr_date, "%Y-%m-%d").date()):
    print("Fetching new info ...")
    df_blk, df_vg = fetch_most_recent_info()
  else:
    df_blk, df_vg = pd.read_excel(f"./out/{out_workbooks[0]}"), pd.read_excel(f"./out/{out_workbooks[1]}")

  return df_blk, df_vg


df_blk, df_vg = handle_new_day()

In [ ]:
def map_yahoo_finance_data():
  data = get_yahoo_finance_data()
  
def map_tipranks_data():
  data = get_tipranks_ratings()